In [1]:
import time

import httpx
import sqlmodel

from arpav_ppcv import (
    config, 
    database,
)
from arpav_ppcv.schemas.static import ObservationStationManager
from arpav_ppcv.observations_harvester import operations

client = httpx.Client()
settings = config.get_settings()
session = sqlmodel.Session(database.get_engine(settings))

arpav_series_configuration = database.get_observation_series_configuration_by_identifier(
    session, 
    "tas-absolute-annual-arpa_v-yearly"
)
arpafvg_series_configuration = database.get_observation_series_configuration_by_identifier(
    session, 
    "tas-absolute-annual-arpa_fvg-yearly"
)

station_arpav = database.get_observation_station_by_code(session, "arpav-129")
station_arpafvg = database.get_observation_station_by_code(session, "arpafvg-10")

print(f"{arpav_series_configuration.identifier=}")
print(f"{station_arpav.code=}")
print(f"{arpafvg_series_configuration.identifier=}")
print(f"{station_arpafvg.code=}")

arpav_series_configuration.identifier='tas-absolute-annual-arpa_v-yearly'
station_arpav.code='arpav-129'
arpafvg_series_configuration.identifier='tas-absolute-annual-arpa_fvg-yearly'
station_arpafvg.code='arpafvg-10'


In [2]:
arpav_raw_measurements = list(
    operations.fetch_arpav_station_measurements(
        client,
        station_arpav,
        arpav_series_configuration,
        settings.arpav_observations_base_url,
    )    
)

In [3]:
arpav_parsed_measurements = [
    operations.parse_arpav_measurement(
        raw_measurement, 
        year_period, 
        station_arpav, 
        arpav_series_configuration.climatic_indicator
    ) for year_period, raw_measurement in arpav_raw_measurements
]

In [6]:
arpafvg_raw_measurements = list(
    operations.fetch_arpafvg_station_measurements(
        client,
        station_arpafvg,
        arpafvg_series_configuration,
        settings.arpafvg_observations_base_url,
        settings.arpafvg_auth_token,
    )
)

In [8]:
arpafvg_parsed_measurements = [
    operations.parse_arpafvg_measurement(
        raw_measurement, 
        year_period, 
        station_arpafvg, 
        arpafvg_series_configuration.climatic_indicator
    ) for year_period, raw_measurement in arpafvg_raw_measurements
]

In [9]:
arpafvg_parsed_measurements

[ObservationMeasurementCreate(value=12.850555555556, date=datetime.date(1995, 1, 1), measurement_aggregation_type=<MeasurementAggregationType.YEARLY: 'yearly'>, observation_station_id=926, climatic_indicator_id=271, identifier='271-19950101/01/95'),
 ObservationMeasurementCreate(value=12.432394366197, date=datetime.date(1996, 1, 1), measurement_aggregation_type=<MeasurementAggregationType.YEARLY: 'yearly'>, observation_station_id=926, climatic_indicator_id=271, identifier='271-19960101/01/96'),
 ObservationMeasurementCreate(value=13.702247191011, date=datetime.date(1997, 1, 1), measurement_aggregation_type=<MeasurementAggregationType.YEARLY: 'yearly'>, observation_station_id=926, climatic_indicator_id=271, identifier='271-19970101/01/97'),
 ObservationMeasurementCreate(value=13.095879120879, date=datetime.date(1998, 1, 1), measurement_aggregation_type=<MeasurementAggregationType.YEARLY: 'yearly'>, observation_station_id=926, climatic_indicator_id=271, identifier='271-19980101/01/98'),
